In [1]:
# imports
import pandas as pd
import csv
import numpy as np; np.random.seed(0)
import os
import glob

In [2]:
# Select the paths of the excel files

file_of_language_1 = pd.read_excel("PATH_TO_YOUR_FILE", header=0, engine='openpyxl')
file_of_language_2 = pd.read_excel("PATH_TO_YOUR_FILE", header=0, engine='openpyxl')

In [3]:
# Select the name of the output file
output_name = "same_glosses"

In [4]:
new_cols = []
for coll in file_of_language_1.columns:
    new_cols.append(coll.replace(" ","_"))
file_of_language_1.columns = new_cols

new_cols2 = []
for coll in file_of_language_2.columns:
    new_cols2.append(coll.replace(" ","_"))
file_of_language_2.columns = new_cols2

In [5]:
# View all the columns of the excel file after they have been imported here (use them in the next step).
# Some of them might be writen differently here compared to your excel file. Use these ones.

print(file_of_language_1.columns)

Index(['GSL_ID-Gloss', 'English_Gloss', 'Initial_HS', 'Final_HS',
       'Initial_Location_', 'Final_Location', 'Unnamed:_6', 'Unnamed:_7',
       'Unnamed:_8', 'Unnamed:_9', 'Unnamed:_10'],
      dtype='object')


In [6]:
# select only the columns that interest you

columns_to_include = ["GSL_ID-Gloss", "Initial_HS", "Final_HS","Initial_Location_", "Final_Location"]

In [7]:
# Select column name that contains the gloss
gloss_column = 'GSL_ID-Gloss'

In [8]:
signs1_df_new = file_of_language_1[columns_to_include].dropna()

signs2_df_new = file_of_language_2[columns_to_include].dropna()

In [9]:
# Distance function
def dict_compare(d1, d2, gloss_column):
    #get keys of d1 that do not have a value == 0
    d1_non_zero_keys = []
    d2_non_zero_keys = []
    
    non_used_fields = []
    # d1_non_gloss_keys = d1.keys
    for i in d1.keys():
        if i != gloss_column:
            d1_non_zero_keys.append(i)
    for j in d2.keys():
         if j != gloss_column:
            d2_non_zero_keys.append(j)
    d1_keys = set(d1_non_zero_keys)
    d2_keys = set(d2_non_zero_keys)
    # shared_keys = d1_keys.intersection(d2_keys)
    shared_keys = d1_keys.intersection(d2_keys)
    modified = {o : (d1[o], d2[o]) for o in shared_keys if d1[o] != d2[o]}
    same = set(o for o in shared_keys if d1[o] == d2[o])
    return modified, same, shared_keys

In [10]:
import re
all_distances_dict = {}
for j in signs1_df_new[gloss_column]:
  ex4 = signs1_df_new[signs1_df_new[gloss_column] == j].to_dict(orient='list')
  ex4_distances_dict = {}
#   print(ex4)
  
  for l in signs2_df_new[gloss_column]:
    l2 = ''.join([w for w in l if not w.isdigit()]).replace("_", "")
    j2 = ''.join([w for w in j if not w.isdigit()]).replace("_", "")
    if j2 in l2:
      ex5 = signs2_df_new[signs2_df_new[gloss_column] == l].to_dict(orient='list')
      modified, same, shared_keys = dict_compare(ex4, ex5, gloss_column)
      distance = len(same)/len(shared_keys)
      ex4_distances_dict[l] = distance
  ex4_distances_dict_sorted = dict(sorted(ex4_distances_dict.items(), key=lambda item: item[1],reverse=True))
  all_distances_dict[j] = ex4_distances_dict_sorted

In [11]:
# all_distances_dict

In [12]:

path = './' 

with open(path+output_name+'.csv', 'w') as f:
    for key in all_distances_dict.keys():
        f.write("%s,%s\n"%(key,all_distances_dict[key]))